# DeepTesla Jr.

In [5]:
from __future__ import print_function

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

#from quiver_engine import server

from PIL import Image
import glob
import os
#import pickle
import numpy as np
#import scipy

batch_size = 32
epochs = 4
#save_dir = os.path.join(os.getcwd(), 'saved_models')
#model_name = 'deeptesla_trained_model.h5'

filelist = glob.glob("/mnt/j/Datasets/deeptesla/training/*.jpg")
x_train = np.array([np.array(Image.open(fname)) for fname in filelist])
filelist = glob.glob("/mnt/j/Datasets/deeptesla/validation/*.jpg")
x_test = np.array([np.array(Image.open(fname)) for fname in filelist])

#x_train("/mnt/j/Datasets/deeptesla/training")
#x_test("/mnt/j/Datasets/deeptesla/validation")

y_train = np.genfromtxt("/mnt/j/Datasets/deeptesla/training.csv", delimiter = ',')
y_test = np.genfromtxt("/mnt/j/Datasets/deeptesla/validation.csv", delimiter = ',')

model = Sequential()
model.add(Conv2D(8, (3, 3), padding = 'same',
                input_shape = x_train[0].shape))
model.add(Activation('relu'))

model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))

#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

#model.add(Conv2D(64, (3, 3), padding='same'))
#model.add(Activation('relu'))

#model.add(Conv2D(8, (3, 3)))
#model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512))
model.add(Activation('tanh'))

model.add(Dropout(0.4))
model.add(Dense(1))
model.add(Activation('linear'))

# The model using RMSprop
model.compile(loss = 'mean_squared_error',
              optimizer = 'adadelta')

# Start Quiver
#server.launch(model, input_folder = "/mnt/j/Datasets/deeptesla/training")

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

model.fit(x_train, y_train,
          batch_size = batch_size,
          epochs = epochs,
          validation_data = (x_test, y_test),
          shuffle = False,
          verbose = 1)

# Save model and weights
#if not os.path.isdir(save_dir):
#    os.makedirs(save_dir)

#model_path = os.path.join(save_dir, model_name)
#model.save(model_path)

#print('Saved trained model at %s ' % model_path)


Train on 21600 samples, validate on 5400 samples
Epoch 1/4
21600/21600 [==============================] - 752s - loss: 5.5340 - val_loss: 16.4877
Epoch 2/4
21600/21600 [==============================] - 751s - loss: 4.2340 - val_loss: 11.8971
Epoch 3/4
21600/21600 [==============================] - 764s - loss: 3.8755 - val_loss: 8.0889
Epoch 4/4
21600/21600 [==============================] - 755s - loss: 3.8169 - val_loss: 5.5370
